In [24]:
!pip install tqdm

In [25]:
import math
import random
import numpy as np
import pandas as pd
import nltk
from tqdm import tqdm
nltk.data.path.append('.')

In [17]:
with open("en_US.twitter.txt", "r",encoding="utf8") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:100])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")


Data type: <class 'str'>
Number of letters: 164456396
First 300 letters of the data
-------


'How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way '

-------
Last 300 letters of the data
-------


'od. I see the success you got poppin in yo area.\nRT : Consumers are visual. They want data at their finger tips. Mobile is the only way to deliver this, 24/7.\nu welcome\nIt is #RHONJ time!!\nThe key to keeping your woman happy= attention, affection, treat her like a queen and sex her like a pornstar!\n'

-------


In [18]:
def split_to_sentences(data):
    """
    Split data by linebreak "\n"
    Args:
    data: str
    Returns:
        A list of sentences
    """
    sentences = data.split("\n")
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 0] 
    return sentences   

In [19]:
#Example
x = "Thunder and lightning.\nEnter three Witches."
split_to_sentences(x)

['Thunder and lightning.', 'Enter three Witches.']

In [26]:
def tokenize_sentences(sentences):
    tokenized_sentences = []
    for sentence in tqdm(sentences):
        sentence = sentence.lower()
        tokenized = nltk.word_tokenize(sentence)
        # append the list of words to the list 
        tokenized_sentences.append(tokenized)
    
    return tokenized_sentences

In [27]:
sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]
tokenize_sentences(sentences)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]


[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green', '.'],
 ['roses', 'are', 'red', '.']]

In [28]:
def get_tokenized_data(data):
    sentences = split_to_sentences(data)
    tokenized_sentences = tokenize_sentences(sentences)
    return tokenized_sentences

In [29]:
tokenized_data = get_tokenized_data(data)

100%|██████████████████████████████████████████████████████████████████████| 2360148/2360148 [04:12<00:00, 9364.66it/s]


In [37]:
random.seed(87)
random.shuffle(tokenized_data)# changes the original list/tuple/string
train_size = int(len(tokenized_data) * 0.8)
train_data = tokenized_data[0:train_size]
test_data = tokenized_data[train_size:]

In [38]:
print("{} data are split into {} train and {} test set".format(len(tokenized_data), len(train_data), len(test_data)))

print("First training sample:")
print(train_data[0])
      
print("First test sample")
print(test_data[0])

2360148 data are split into 1888118 train and 472030 test set
First training sample:
['yep', '!', 'about', 'a', 'year', 'ago', 'now', 'i', 'moved', 'down', 'to', 'ca', '.', 'i', 'like', 'it', '!']
First test sample
['q.', 'what', "'s", 'the', 'definition', 'of', 'a', 'quarter', 'tone', '?', 'a.', 'a', 'bagpiper', 'tuning', 'his', 'drones', '.']


In [39]:
def count_words(tokenized_sentences):   
    word_counts = {}
    for sentence in range(len(tokenized_sentences)): 
        # go through tokens in the sentence
        for token in (tokenized_sentences[sentence]): 
            if token not in word_counts.keys(): 
                word_counts[token] = 1
            else:
                word_counts[token] += 1
    return word_counts

In [40]:
def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    # appear at least 'minimum_freq' times.
    closed_vocab = []
    word_counts = count_words(tokenized_sentences)
    for word, cnt in word_counts.items(): 

        if cnt>=count_threshold:
            closed_vocab.append(word)
    return closed_vocab

In [41]:
def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)

    replaced_tokenized_sentences = []
    
    # go through sentences
    for sentence in tokenized_sentences:
        replaced_sentence = []
        # go through tokens
        for token in sentence: 
            # Is the token in the closed vocabulary?
            if token in vocabulary:
                # If so, append the word to the replaced_sentence
                replaced_sentence.append(token)
            else:
                # otherwise, append the unknown token 
                replaced_sentence.append(unknown_token)
        replaced_tokenized_sentences.append(replaced_sentence)    
    return replaced_tokenized_sentences


In [43]:
def preprocess_data(train_data, test_data, count_threshold):
    vocabulary = get_words_with_nplus_frequency(train_data,count_threshold)
    train_data_replaced=[]
    test_data_replaced=[]
    # replace less common words with "<unk>" training set
    for sentence in tqdm(range(len(train_data))):
        parole=[]
        for word in range(len(train_data[sentence])):
            if train_data[sentence][word] in vocabulary:
                parole.append(train_data[sentence][word])
            else:
                parole.append("<unk>")
                
        train_data_replaced.append(parole)
    # replace less common words with "<unk>" test set
    for sentence in tqdm(range(len(test_data))):
        parole_test=[]
        for word in range(len(test_data[sentence])):
            if test_data[sentence][word] in vocabulary:
                parole_test.append(test_data[sentence][word])
            else:
                parole_test.append("<unk>")
        test_data_replaced.append(parole_test)
    return train_data_replaced, test_data_replaced, vocabulary

In [44]:
minimum_freq = 2
train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data,minimum_freq)


100%|█████████████████████████████████████████████████████████████████████████| 472030/472030 [24:17<00:00, 323.95it/s]


In [45]:
print("First preprocessed training sample:")
print(train_data_processed[0])
print()
print("First preprocessed test sample:")
print(test_data_processed[0])
print()
print("First 10 vocabulary:")
print(vocabulary[0:10])
print()
print("Size of vocabulary:", len(vocabulary))

First preprocessed training sample:
['yep', '!', 'about', 'a', 'year', 'ago', 'now', 'i', 'moved', 'down', 'to', 'ca', '.', 'i', 'like', 'it', '!']

First preprocessed test sample:
['q.', 'what', "'s", 'the', 'definition', 'of', 'a', 'quarter', 'tone', '?', 'a.', 'a', 'bagpiper', 'tuning', 'his', 'drones', '.']

First 10 vocabulary:
['yep', '!', 'about', 'a', 'year', 'ago', 'now', 'i', 'moved', 'down']

Size of vocabulary: 146227


In [46]:
def count_n_grams(data, n, start_token='<s>', end_token = '<e>'):
    n_grams = {}
    for sentence in range(len(data)):
        sentences = [start_token]*n+list(data[sentence])+[end_token]
        sentences = tuple(sentences)
        for i in range(0,len(sentences)-n+1):
            n_gram = sentences[i:i+n]
           # Is the n-gram in the dictionary?
            if n_gram in n_grams.keys():
                n_grams[n_gram] += 1
            else:
               # otherwise is 1
                n_grams[n_gram] = 1
    return n_grams

In [47]:
#Example
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
print("Uni-gram:")
print(count_n_grams(sentences, 1))
print("Bi-gram:")
print(count_n_grams(sentences, 2))

Uni-gram:
{('<s>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<e>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bi-gram:
{('<s>', '<s>'): 2, ('<s>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<e>'): 2, ('<s>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


In [48]:
def estimate_probability(word, previous_n_gram, 
                         n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    if type(previous_n_gram)==list:
        previous_n_grams = tuple(previous_n_gram)
    else:
        previous_n_grams = tuple([previous_n_gram])
    previous_n_gram_count = n_gram_counts.get(previous_n_grams,0)
    # and apply k-smoothing
    denominator = previous_n_gram_count+vocabulary_size*1
    n_plus1_gram =previous_n_grams+tuple([word])
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram,0)
        
    # and apply smoothing
    numerator = n_plus1_gram_count+k
    probability = numerator/denominator
    return probability

In [49]:
def estimate_probabilities(previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0):
 
    previous_n_gram = previous_n_gram
    # add <e> <unk> to the vocabulary <s> is not needed since it should not appear as the next word
    vocabulary = vocabulary + ["<e>", "<unk>"]
    vocabulary_size = len(vocabulary)
    probabilities = {}
    for word in vocabulary:
        probability = estimate_probability(word, previous_n_gram, 
                                           n_gram_counts, n_plus1_gram_counts, 
                                           vocabulary_size, k=k)
        probabilities[word] = probability
    return probabilities

In [50]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    vocabulary = vocabulary + ["<e>", "<unk>"]
    n_grams = []
    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1]
        n_grams.append(n_gram)
    n_grams = list(set(n_grams))
    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    col_index = {word:j for j, word in enumerate(vocabulary)}
    nrow = len(n_grams)
    ncol = len(vocabulary)
    count_matrix = np.zeros((nrow, ncol))
    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count
    
    count_matrix = pd.DataFrame(count_matrix, index=n_grams, columns=vocabulary)
    return count_matrix

In [51]:
def make_probability_matrix(n_plus1_gram_counts, vocabulary, k):
   count_matrix = make_count_matrix(n_plus1_gram_counts, unique_words)
   count_matrix += k
   prob_matrix = count_matrix.div(count_matrix.sum(axis=1), axis=0)
   return prob_matrix

In [52]:
def calculate_perplexity(sentences, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):

   n = len(list(n_gram_counts.keys())[0]) 
   sentence = ["<s>"] * n + sentences + ["<e>"]

   sentence = tuple(sentence)
   N = len(sentence)
   product_pi = 1.0
   for t in range(n, N): # complete this line

       n_gram = sentence[t-n]
       word = sentence[t]
       probability = estimate_probability(word, n_gram, 
                        n_gram_counts, n_plus1_gram_counts, vocabulary_size, k)
       
    
       product_pi *= 1/probability

   # Nth root of the product
   perplexity = product_pi**(1/float(N))
   perplexity = float(perplexity)

   return perplexity

In [55]:
def suggest_a_word(previous_tokens, n_gram_counts, n_plus1_gram_counts, vocabulary, k=1.0, start_with=None):

   n = len(list(n_gram_counts.keys())[0]) 
   probabilities = estimate_probabilities(previous_tokens,
                                          n_gram_counts, n_plus1_gram_counts,
                                          vocabulary, k=k)
   suggestion = None    
   max_prob = 0
   for word, prob in probabilities.items(): 
       if start_with != None : # complete this line
           if word.startswith(start_with)==False: 
               #If so, move onto the next word
               continue 
       if prob>max_prob: 
           suggestion = word
           max_prob = prob

   return suggestion, max_prob

In [56]:
#Example
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)

previous_tokens = ["i", "like"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'i like',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")

print()
#Example starts_with
tmp_starts_with = 'c'
tmp_suggest2 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0, start_with=tmp_starts_with)
print(f"The previous words are 'i like', the suggestion must start with `{tmp_starts_with}`\n\tand the suggested word is `{tmp_suggest2[0]}` with a probability of {tmp_suggest2[1]:.4f}")

The previous words are 'i like',
	and the suggested word is `like` with a probability of 0.1111

The previous words are 'i like', the suggestion must start with `c`
	and the suggested word is `cat` with a probability of 0.1111


In [57]:
 def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    model_counts = len(n_gram_counts_list)
    suggestions = []
    for i in range(model_counts-1):
        n_gram_counts = n_gram_counts_list[i]
        n_plus1_gram_counts = n_gram_counts_list[i+1]
        
        suggestion = suggest_a_word(previous_tokens, n_gram_counts,
                                    n_plus1_gram_counts, vocabulary,
                                    k=k, start_with=start_with)
        suggestions.append(suggestion)
    return suggestions

In [58]:
n_gram_counts_list = []
for n in range(1, 6):
    print("Computing n-gram counts with n =", n, "...")
    n_model_counts = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_model_counts)
    

Computing n-gram counts with n = 1 ...
Computing n-gram counts with n = 2 ...
Computing n-gram counts with n = 3 ...
Computing n-gram counts with n = 4 ...
Computing n-gram counts with n = 5 ...


In [59]:
previous_tokens = ["i", "am", "to"]
tmp_suggest1 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest1)

The previous words are ['i', 'am', 'to'], the suggestions are:


[('yep', 6.83858878881754e-06),
 ('yep', 6.83858878881754e-06),
 ('have', 9.567089213106912e-05),
 ('yep', 6.83858878881754e-06)]

In [60]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest2 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest2)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('yep', 6.83858878881754e-06),
 ('yep', 6.83858878881754e-06),
 ('yep', 6.83858878881754e-06),
 ('?', 0.00017088992638061973)]

In [61]:
previous_tokens = ["hey", "how", "are", "you"]
tmp_suggest3 = get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with="d")

print(f"The previous words are {previous_tokens}, the suggestions are:")
display(tmp_suggest3)

The previous words are ['hey', 'how', 'are', 'you'], the suggestions are:


[('down', 6.83858878881754e-06),
 ('down', 6.83858878881754e-06),
 ('down', 6.83858878881754e-06),
 ('doing', 6.15203734970231e-05)]